In [ ]:
pip install -i https://mirrors.aliyun.com/pypi/simple/ --upgrade selenium==3.141.0

In [ ]:
!pip install pymysql

In [1]:
#webdriver用于生成浏览器
from selenium import webdriver
#Options用于初始化浏览器
from selenium.webdriver.chrome.options import Options
#用于搜索栏输入后确定。
from selenium.webdriver.common.keys import Keys
#pymysql是MySQL的接口
import pymysql.cursors
#用于计算本次爬虫用时/非必要/
import datetime
#用于selenium的强制等待
import time

from selenium.webdriver.common.by import By

# cookie 缓存登陆
import json
from time import sleep
from selenium import webdriver
import pandas as pd

In [18]:
class Spider:
    phoneList = []
    #浏览器的头
    headers={"User-Agent": "Mozilla/5.0 (Windows; U; Windows NT 6.0 x64; en-US; rv:1.9pre) Gecko/2008072421 Minefield/3.0.2pre"}
    #headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
    
        #初始化
    def startUp(self, url):
        # Initializing Chrome browser
        chrome_options = Options()
        #下面两项为无头与禁止gpu加速；
        #--headless比较重要，执行selenium时不会弹出浏览器窗口
        #chrome_options.add_argument('--headless')
        #chrome_options.add_argument('--disable-gpu')
        
        #禁止图片和css加载
        #prefs = {"profile.managed_default_content_settings.images": 2,'permissions.default.stylesheet':2}
        #chrome_options.add_experimental_option("prefs", prefs)

        #创建Chrome浏览器
        self.browser = webdriver.Chrome(chrome_options = chrome_options)
        
        # 访问网站，清空旧cookies信息
        self.browser.get(url)  
        sleep(1)
        self.browser.delete_all_cookies()
 
        # 加载 cookies信息
        with open("cookies.txt", "r") as f:
            cookies = json.load(f)
        for cookie in cookies:
            self.browser.add_cookie(cookie)
        #重新打开
        self.browser.get(url)
        
        stopX = input('是否继续：')

        #这里是selenium是使用元素方法，输入与确认等
        #keyInput = self.browser.find_element_by_id("key")
        #keyInput.send_keys(key)
        #keyInput.send_keys(Keys.ENTER)
        
    #主要过程
    def processSpider(self,Page,page):
        try:
        	#强制等待1秒
            time.sleep(1)
            print(self.browser.current_url)
            
            goodsList = self.browser.find_elements_by_xpath("//div[@id='J_goodsList']//li[@class='gl-item']")
            for goods in goodsList:
                phoneData = {}
                #同理很容易找到所需要数据的位置，执行xpath和text()得到字符串内容
                price = goods.find_element_by_xpath(".//div[@class='p-price']//i").text
                noteItem = goods.find_element_by_xpath(".//div[@class='p-name p-name-type-2']//em")
                note = noteItem.text
                
                #观察发现有部分数据标题前是‘京品手机’等无关数据，通过if语句剔除。
                if note[0:2] == '京品':
                    note = note[5:]
                if '】' in note:
                    note = note[note.index('】')+1 :]
               
                
                phoneData["价格"] = price
                phoneData["标题"] = note
                
                noteItem.click()
                sleep(1)
                handles = self.browser.window_handles
                self.browser.switch_to_window(handles[1])
                
                comment = self.browser.find_element_by_xpath("//div[@id='comment-count']//a").text
                phoneData["评论总量"] = comment
                
                self.browser.find_element_by_xpath("//div[@id='detail']//li[contains(text(),'规格与包装')]").click()
                sleep(1)
                ptableList = self.browser.find_elements_by_xpath(".//div[@class='Ptable']//dl[@class='clearfix']")

                for item in ptableList:
                    itemTemp =  item.text.split("\n")
                    phoneData[itemTemp[0]] = itemTemp[1]
                              
                print(page)
                print(phoneData)  
                
                self.phoneList.append(phoneData)
                
                self.browser.close()
                self.browser.switch_to_window(handles[0])
                
            try:
                self.browser.find_element_by_xpath("//span[@class='p-num']//a[@class='pn-next disabled']")             
                newPage = input('该轮爬虫已完成是否继续，继续请输出页数：')
                self.processSpider(newPage,0)
            except:
                print("运行了pn-next！！！")
                nextPage = self.browser.find_element_by_xpath("//span[@class='p-num']//a[@class='pn-next']")
                nextPage.click()
                time.sleep(3) 
                #当前页面数+1，以下if语句用来限制翻页数|。
                page += 1
                if page == Page : 
                    return
                self.processSpider(Page,page)    
                
        except Exception as err:
            print(err)
            newPage = input('中断爬虫继续按-1，退出按0：')
            if newPage == "-1":
                self.processSpider(newPage,0)
            return
            
        
            #执行爬虫，即主程序
    def executeSpider(self, url):
        starttime = datetime.datetime.now()
        print("Spider starting......")
        Page = int(input('请输入要爬取的页数,如若不限制,请输入-1：'))
        self.startUp(url)
        self.processSpider(Page, 0)
        print("Spider completed......")
        endtime = datetime.datetime.now()
        elapsed = (endtime - starttime).seconds
        print("Total ", elapsed, " seconds elapsed")
        

In [36]:
url = "https://search.jd.com/search?keyword=%E6%89%8B%E6%9C%BA&qrst=1&wq=%E6%89%8B%E6%9C%BA&shop=1&ev=exbrand_Apple%5E&pvid=13411f2e565849e9b88733786f4bca12&cid3=655&cid2=653&stock=1&click=1"
#spider = Spider()
phoneList = spider.executeSpider(url)

Spider starting......
请输入要爬取的页数,如若不限制,请输入-1：-1


C:\Users\Li Rui\AppData\Local\Temp\ipykernel_68776\2876089523.py:21: DeprecationWarning: use options instead of chrome_options
  self.browser = webdriver.Chrome(chrome_options = chrome_options)


是否继续：-1
https://search.jd.com/search?keyword=%E6%89%8B%E6%9C%BA&qrst=1&wq=%E6%89%8B%E6%9C%BA&shop=1&stock=1&ev=exbrand_%E5%8D%8E%E4%B8%BA%EF%BC%88HUAWEI%EF%BC%89%5E&pvid=7033bc6901d84f3d8ac175754535ed7f&cid3=655&cid2=653&page=37&s=1081&click=0


C:\Users\Li Rui\AppData\Local\Temp\ipykernel_68776\2876089523.py:71: DeprecationWarning: use driver.switch_to.window instead
  self.browser.switch_to_window(handles[1])


0
{'价格': '1299.00', '标题': '优畅享20手机 全网通5G版 5000mAh大电池 6.6英寸影音大屏 双卡双待* 绮境森林 6GB+128GB', '评论总量': '39', '机型': 'U-Magic 20', '入网型号': '-', '机身尺寸': '宽76mm；长165.2mm；厚9.2mm', 'CPU型号': '联发科 天玑720', '机身重量': '188g', '屏幕特色': '莱茵护眼；无', '屏幕分辨率': '1600*720', '屏幕材质': 'lcd', '屏幕刷新率': '以官网信息为准', '屏幕尺寸': '6.6英寸', '无线充电': '不支持', '系统': 'Android', '4G网络': '4G FDD-LTE；4G TD-LTE', 'SIM卡数量': '2个', 'SIM卡类型': 'Nano SIM', '双卡机类型': '双卡双待', '5G网络': '支持5G', '3G/2G网络': '以官网信息为准', '充电接口': 'Type-C', '数据接口': 'WIFI；蓝牙', '耳机接口': '3.5mm', '生物识别': '侧面指纹', '后摄主像素': '1300万像素', '前摄主像素': '800万像素', '后摄3-tele像素': '普通(以官网信息为准)', '后摄2-超广角像素': '普通(以官网信息为准)'}


...

0
{'价格': '2439.00', '标题': '黑鲨5 RS 骁龙888\\/888+ 逆重力VC液冷系统 磁动力升降肩键 144Hz屏 5G游戏手机 天穹黑 8GB+256GB 骁龙888', '评论总量': '100+', '机型': '黑鲨 5 RS', '上市日期': '2022-03-29', '入网型号': 'SHARK KSR-A0', '机身尺寸': '宽76.21mm；长163.69mm；厚9.9mm', 'CPU型号': '其他', '机身颜色': '天穹黑', '机身重量': '220g', '存储卡': '不支持', '机身内存': '256GB', '屏幕刷新率': '144Hz', '屏幕特色': '泰尔实验室认证；SGS认证', '屏幕材质': 'OLED', '屏幕尺寸': '6.67英寸', '无线充电': '不支持', '充电功率': '120W', '电池容量': '4500mAh(typ)', '系统': 'Android', '3G/2G网络': '2G GSM(移动/联通)；3G TD-SCDMA(移动)；3G CDMA2000(电信)；3G WCDMA(联通)', '双卡机类型': '双卡双待', 'SIM卡数量': '2个', 'SIM卡类型': 'Nano SIM', '4G网络': '4G FDD-LTE；4G TD-LTE', '5G网络': '支持5G', '充电接口': 'Type-C', '数据接口': 'WIFI；蓝牙', '耳机接口': '3.5mm', '生物识别': '侧面指纹；人脸识别', '后摄2-超广角像素': '800万像素', '后摄3-tele像素': '普通(以官网信息为准)', '后摄主像素': '6400万像素'}
0
{'价格': '2799.00', '标题': '黑鲨5 骁龙870 120W超级闪充 磁动力升降肩键 144Hz AMOLED电竞屏 5G游戏手机 曙光白 12GB+256GB 高能版', '评论总量': '1000+', '机型': '黑鲨 5', '入网型号': 'SHARK PAR-A0', '上市日期': '2022-03-30', '机身尺寸': '宽76.25mm；长163.83mm；厚10.37mm', 'CPU型号': '骁龙870', '机

0
{'价格': '4999.00', '标题': '中兴Axon 40 Ultra 屏下摄像 三主摄计算摄影 高通骁龙8 80W闪充 12+256G 敦煌 官方标配', '评论总量': '100+', '机型': '中兴 Axon 40 Ultra', '入网型号': 'A2023P', '机身尺寸': '宽73.5mm；长163.2mm；厚8.4mm', 'CPU型号': '骁龙8 Gen 1', '机身重量': '204g', '存储卡': '不支持', '屏幕特色': '无', '屏幕材质': 'OLED曲面屏', '屏幕分辨率': '2400*1080', '屏幕尺寸': '6.8英寸', '屏幕刷新率': '120Hz', '无线充电': '不支持', '系统': 'Android', '5G网络': '支持5G', 'SIM卡类型': '以官网信息为准', '4G网络': '4G FDD-LTE；4G TD-LTE', '3G/2G网络': '以官网信息为准', '双卡机类型': '以官网信息为准', 'SIM卡数量': '2个', '数据接口': '无', '耳机接口': 'Type-C', '充电接口': 'Type-C', '生物识别': '人脸识别', '后摄3-tele像素': '普通(以官网信息为准)', '后摄主像素': '6400万像素', '前摄主像素': '1600万像素', '后摄2-超广角像素': '普通(以官网信息为准)'}
0
{'价格': '2029.00', '标题': 'ZTE 中兴Axon 30/30S 骁龙870 吴京代言 屏下摄像手机AMOLED 12+256G 夜幕黑【Axon30S】 官方标配【享90天碎屏宝】', '评论总量': '1000+', '机型': '中兴 Axon 30', '入网型号': 'ZTE A2322', 'CPU型号': '骁龙870', '机身尺寸': '宽77.8mm；长170.2mm；厚7.8mm', '存储卡': '不支持', '屏幕材质': '柔性直屏', '屏幕尺寸': '6.92英寸', '屏幕分辨率': '2460*1080', '屏幕刷新率': '120Hz', '屏幕特色': '无', '无线充电': '不支持', '系统': 'Android', '双卡机类型'

0
{'价格': '839.00', '标题': '中兴 远航10 全网通5G智能手机 单位用机 碧波蓝 6GB+128GB', '评论总量': '45', '机型': '中兴 远航10', '入网型号': 'ZTE 7530N', '机身尺寸': '宽75.5mm；长163.3mm；厚8.2mm', 'CPU型号': '天机700', '机身重量': '1.7g', '存储卡': '不支持', '屏幕尺寸': '6.52英寸', '屏幕刷新率': '90Hz', '屏幕材质': 'LCD', '屏幕特色': '无', '屏幕分辨率': '720P', '电池可拆卸': '不可拆卸', '无线充电': '不支持', '系统': 'Android', '3G/2G网络': '2G GSM(移动/联通)；3G WCDMA(联通)', '4G网络': '4G FDD-LTE；4G TD-LTE', '5G网络': '支持5G', 'SIM卡数量': '2个', 'SIM卡类型': 'Nano SIM', '双卡机类型': '双卡双待', '充电接口': 'Type-C', '耳机接口': 'Type-C', '数据接口': 'WIFI；蓝牙；OTG接口', '生物识别': '侧面指纹', '后摄2-超广角像素': '普通(以官网信息为准)', '后摄主像素': '1600万像素', '后摄3-tele像素': '无', '前摄主像素': '800万像素'}
0
{'价格': '2399.00', '标题': 'ZTE 中兴Axon30 全新一代屏下摄像手机AMOLED120HZ屏 黑曜 12GB+256GB 官方标配', '评论总量': '56', '机型': '中兴 Axon 30', '入网型号': 'ZTE A2322', '机身尺寸': '宽77.8mm；长170.2mm；厚7.8mm', '机身重量': '189g', '屏幕尺寸': '6.92英寸', '屏幕特色': '无', '屏幕刷新率': '120Hz', '无线充电': '不支持', '系统': 'Android', '3G/2G网络': '2G GSM(移动/联通)；3G WCDMA(联通)', 'SIM卡数量': '2个', '5G网络': '支持5G', '双卡机类型': '双卡双待', '4G

0
{'价格': '6599.00', '标题': '索尼（SONY）Xperia 5 IV 智能5G手机 6.1英寸HDR OLED直屏 5000mAh电池 8G+256G 湖绿', '评论总量': '100+', '上市日期': '2022-10-10', '入网型号': 'XQ-CQ72', '机型': '索尼 Xperia5 III', 'CPU型号': '高通骁龙8 Gen 1 移动平台', '机身颜色': '青山翠', '机身尺寸': '宽156mm；长67mm；厚8.2mm', '机身重量': '172g', '存储卡': '不支持', '机身内存': '256GB', '屏幕刷新率': '120Hz', '屏幕尺寸': '6.1英寸', '屏幕分辨率': '1080x2520', '屏幕材质': '康宁® 大猩猩® VictusTM 玻璃', '屏幕特色': '无', '充电功率': '30W', '无线充电': '以官网信息为准', '系统': 'Android', '5G网络': '支持5G', '3G/2G网络': '以官网信息为准', '双卡机类型': '双卡双待', 'SIM卡数量': '2个', 'SIM卡类型': '以官网信息为准；Nano SIM', '4G网络': '4G TD-LTE', '数据接口': 'WIFI；蓝牙；NFC', '充电接口': 'Type-C', '耳机接口': '3.5mm', '生物识别': '侧面指纹', '后摄2-超广角像素': '1200万像素', '后摄主像素': '1200万像素', '后摄3-tele像素': '1200万像素', '前摄主像素': '1200万像素'}
0
{'价格': '9890.00', '标题': '爱心东东\n索尼 Xperia PRO-I 旗舰5G手机 相机摄影 可变机械光圈 4K高刷屏 12G+512G Vlog拍摄套装 GP-VPT2BT蓝牙+监视器 12GB+512GB 黑色', '评论总量': '50', '入网型号': 'XQ-BE72', '机身重量': '211g'}
0
{'价格': '7999.00', '标题': '爱心东东\n索尼（SONY）Xperia 1 IV 智能5G旗舰手机 4K HDR 120Hz OLED屏 暮霞紫 12+512GB

0
{'价格': '3999.00', '标题': '联想拯救者Y90 电竞游戏手机 全新骁龙8 霜刃散热 四肩键双压感 144Hz AMOLED电竞屏 16GB+256GB', '评论总量': '1', '机型': '联想 拯救者电竞手机 Y90', 'CCC强制性认证': '是', '电信设备进网许可证': '是', '入网型号': 'lenovo L71061', '上市日期': '2022-03-10', '机身尺寸': '宽78.4mm；长177mm；厚10.14mm', 'CPU型号': '骁龙8 Gen 1', '机身重量': '252g', '机身颜色': '钛晶灰', '存储卡': '不支持', '屏幕刷新率': '144Hz', '屏幕特色': '莱茵护眼', '屏幕尺寸': '6.92英寸', '屏幕分辨率': '2460*1080', '屏幕材质': 'AMOLED', '无线充电': '不支持', '充电功率': '68W', '系统': 'Android', '4G网络': '4G FDD-LTE；4G TD-LTE', 'SIM卡数量': '2个', 'SIM卡类型': '以官网信息为准', '双卡机类型': '以官网信息为准', '3G/2G网络': '3G TD-SCDMA(移动)；3G CDMA2000(电信)；3G WCDMA(联通)；2G CDMA(电信)', '5G网络': '支持5G', '耳机接口': 'Type-C', '充电接口': 'Type-C', '数据接口': 'WIFI；蓝牙；NFC；OTG接口', '生物识别': '屏幕指纹；人脸识别', '后摄3-tele像素': '无', '后摄主像素': '6400万像素', '后摄2-超广角像素': '1300万像素', '前摄主像素': '1600万像素'}
Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[@class='p-name p-name-type-2']//em"}
  (Session info: chrome=107.0.5304.88)

中断爬虫继续按-1，退出按0：0
Spider completed......

In [37]:
print(len(spider.phoneList))

3584


In [28]:
saveList = spider.phoneList.copy

In [38]:
pf = pd.DataFrame(spider.phoneList)

In [39]:
with pd.ExcelWriter('Iphone.xlsx',options={'encoding':'utf-8'}) as writer:
    pf.to_excel(writer, sheet_name='JD_Phone')

C:\Users\Li Rui\AppData\Local\Temp\ipykernel_68776\1500044750.py:1: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  with pd.ExcelWriter('Iphone.xlsx',options={'encoding':'utf-8'}) as writer:
